# Approach Plate Parsing
Parsing of approach plates to create table of RNAV alititude and visisiblity requirements as well as airport meta data.

In [17]:
from pypdf import PdfReader

reader = PdfReader('./approach_plates/05251R14.PDF')

print(len(reader.pages))

page = reader.pages[0]

text = page.extract_text()
print(text)

1
P
P
A532
14
5498 X 100
0.5% UPWINCHESTER, VIRGINIA
(OKV)AL-5251 (FAA)
WINCHESTER RGNL
WINCHESTER, VIRGINIA(OKV) WINCHESTER RGNLRNAV (GPS) RWY 14
RNAV (GPS) RWY 14CLNC DEL
126.15AWOS-3
124.85 120.45  306.925UNICOM
122.9755498
726
726
(CTAF)POTOMAC APP CON
ELEVApt ElevTDZERwy Idg 
144°70712
W14AWAAS
CH    APP CRS
726
3700 CLADD
LNAV MDA
CIRCLINGMSA RW14 25 NM 
4700
RW14(3.6)107°3700
144°(4.3) 3500
CLADD
324°144°4 NM
RW143700
2700BONCIEMNAR
3500 * LNAV only*
*
4.3 NM144°
HIRL Rwy 14-32REIL Rwy 14TDZE7262844
2639
DALPV DA
VNAVLNAV/334 (400-1) 1060-1181060-1 334 (400-1   )18
1140-1 414 (500-1) 1140-11818414 (500-1   )
1180-1
454 (500-1)1220-1
494 (500-1) 654 (700-1   )12 1480-2
754 (800-2   )1234976- 250 (300-   )34BIPME
2000RW142 NM toWUSBI
to RW141.2 NM
*1400RW143.9 NM toWOGDI
1.2 NM 0.8 NM 1.9 NM 2.2 NM 2.6 NM     MUMSY on V44 eastbound.Procedure NA for arrivals at
     SOGBE on V4 westbound.Procedure NA for arrivals atZENDIACELO
RW143.9 NM toWOGDI RW142 NM toWUSBIEMNAR
SOGBE(IAF)BONCI

In [3]:
import os
import fitz
count = 1
approach_plate_dir = "./approach_plates/"
for filename in os.listdir(approach_plate_dir):
    doc = fitz.open(approach_plate_dir + "/" + filename)
    for page in doc:
    #page = doc.getPage(0)
        text = page.get_text()
        location = text.find("TDZE")
        print(location)

    """  out = open(filename[:-4] + ".txt", "wb")
    for page in doc:
        text = page.get_text(sort=True).encode("utf8")
        out.write(text)
        out.write(bytes((12,)))
    out.close() """


    count += 1
    if count > 2:
        break



438
409


In [18]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

fp = open('test_plate_2.PDF', 'rb')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.get_pages(fp)

for page in pages:
    print('Processing next page...')
    interpreter.process_page(page)
    layout = device.get_result()
    for lobj in layout:
        if isinstance(lobj, LTTextBox):
            x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()
            #if "TDZE" in text:
            print('At %r is text: %s' % ((x, y), text))

Processing next page...
At (18.2111, 562.203521) is text: DREW, MISSISSIPPI

At (173.9938, 562.203521) is text: AL-5859 (FAA)

At (351.7608, 562.203521) is text: 19115

At (20.6755, 548.102321) is text: APP CRS
   

At (25.0844, 540.578641) is text: 180°

At (50.6129, 550.5499209999999) is text: Rwy Idg 
TDZE
Apt Elev

At (81.5853, 552.3128409999999) is text: 3000
137
137

At (247.1752, 550.283922) is text: RNAV (GPS) RWY 18
(M37)

At (290.9747, 536.978013) is text: RULEVILLE-DREW

At (24.0431, 523.3166209999999) is text: T

At (23.1783, 513.692821) is text: A

At (30.5931, 513.692821) is text: NA

At (54.4647, 523.3139209999999) is text: DME/DME RNP-0.3 NA.  Use Cleveland altimeter setting; when not received, 
use Greenwood altimeter setting and increase all MDA 40 feet and increase 
LNAV and Circling Cat C visibility     mile.  Procedure NA at night.  
Helicopter visibility reduction below 1 SM NA.

At (143.7113, 509.45702) is text: 1

At (146.0149, 507.22531999999995) is text: 4

At

In [15]:
import fitz

# Open the PDF file
pdf = fitz.PdfReader("hello.pdf")

# Get the first page
page = pdf.getPage(0)

# Get the text on the page
text = page.extractText()

# Find the coordinates of the word "hello"
coordinates = text.find("hello")

# Print the coordinates
print(coordinates)


AttributeError: module 'fitz' has no attribute 'PdfReader'

In [17]:
from pathlib import Path
from typing import Iterable, Any

from pdfminer.high_level import extract_pages


def show_ltitem_hierarchy(o: Any, depth=0):
    """Show location and text of LTItem and all its descendants"""
    if depth == 0:
        print('element                        x1  y1  x2  y2   text')
        print('------------------------------ --- --- --- ---- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '
        f'{get_optional_bbox(o)} '
        f'{get_optional_text(o)}'
    )

    if isinstance(o, Iterable):
        for i in o:
            show_ltitem_hierarchy(i, depth=depth + 1)


def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of LTItem"""
    return '  ' * depth + o.__class__.__name__


def get_optional_bbox(o: Any) -> str:
    """Bounding box of LTItem if available, otherwise empty string"""
    if hasattr(o, 'bbox'):
        return ''.join(f'{i:<4.0f}' for i in o.bbox)
    return ''


def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):
        return o.get_text().strip()
    return ''


path = Path('~/Downloads/simple1.pdf').expanduser()

pages = extract_pages(path)
show_ltitem_hierarchy(pages)

ImportError: cannot import name 'HOCRConverter' from 'pdfminer.converter' (c:\Python39\lib\site-packages\pdfminer\converter.py)